# Classification task

Hi! It's a classification task baseline notebook.
It include a data reader, baseline model and submission generator.

You should use GPU to train your model, so we recommend using [Kaggle Notebooks](https://www.kaggle.com/docs/notebooks).
To get maximum score of the task, your model should have accuracy greater than `85%`.

You can use everything, that suits into the rules in `README.md`.

In [ ]:
!pip install catalyst optuna ttach torch-lr-finder
!pip install --upgrade wandb
!pip install -U albumentations
#!wandb login 03248ab38d989b0a18ea64ce321cb8ab13a801e6

In [ ]:
!gdown https://drive.google.com/uc?id=1xD8Qx33LeefTXe_KNzERM3TdA4r-UZkA&export=download
!gdown https://drive.google.com/uc?id=1WWiuL8sXlMoBnpbkbqv3tDFgR1Rk_nPE&export=download
!unzip train.zip -d train
!unzip test.zip -d test

In [ ]:
from pathlib import Path
from datetime import datetime
import numpy as np
import cv2

import matplotlib.pyplot as plt
from glob import glob
import random
import os

import torch
import torch.nn as nn
from torch.utils import data
import torchvision

import catalyst
from catalyst import dl
from catalyst.utils import metrics, set_global_seed

In [ ]:
set_global_seed(42)

### Make extra images

In [ ]:
#This function is not necessary, but can boost perfomance
def make_extra_images(image_roots):
    """Function will make extra pictures with horizontal and vertical reflection.
    """

    print('Extra pictures generation started...', end='')
    prefix_names = ['_090', '_180', '_270']

    for path in image_roots:
        files = os.listdir(path)
        files = list(filter(lambda x: x.endswith('.jpg') and not any([pr in x for pr in prefix_names]), files))

        for i, file in enumerate(files):
            img = cv2.imread(os.path.join(path,file))
            # Make extra pictures: flip each of originals photo to 90, 180 and 270 degrees
            for i, angle in enumerate([cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_180, cv2.ROTATE_90_COUNTERCLOCKWISE]):
                img = cv2.rotate(img, angle)
                img_name = os.path.join(path, file[:file.find('.')] + prefix_names[i] + file[file.find('.'):])
                if not os.path.exists(img_name):
                    cv2.imwrite(img_name, img)

In [ ]:
make_extra_images(image_roots=['/content/train/c1d6f6c4',
                               '/content/train/c1d6fa84',
                               '/content/train/c1d6fc6e',
                               '/content/train/c1d6fd90',
                               '/content/train/c1d6fe94',
                               '/content/train/c1d6ff98',
                               '/content/train/c1d70092',
                               '/content/train/c1d70196',
                               '/content/train/c1d702ae',
                               '/content/train/c1d70420'])

Extra pictures generation started...

## Dataset

This code will help you to generate dataset. If your data have the following folder structure:

```
dataset/
    class_1/
        *.ext
        ...
    class_2/
        *.ext
        ...
    ...
    class_N/
        *.ext
        ...
```
First of all `create_dataset` function goes through a given directory and creates a dictionary `Dict[class_name, List[image]]`.
Then `create_dataframe` function creates typical `pandas.DataFrame` for further analysis.
After than `prepare_dataset_labeling` creates a numerical label for each unique class name.
Finally, to add a column with a numerical label value to the DataFrame, we can use `map_dataframe` function.

Additionaly let's save the `class_names` for further usage.

In [ ]:
from catalyst.utils import (
    create_dataset, create_dataframe, get_dataset_labeling, map_dataframe
)

dataset = create_dataset(dirs=f"train/*", extension="*.jpg")
df = create_dataframe(dataset, columns=["class", "filepath"])

tag_to_label = get_dataset_labeling(df, "class")
class_names = [
    name for name, id_ in sorted(tag_to_label.items(), key=lambda x: x[1])
]

df_with_labels = map_dataframe(
    df, 
    tag_column="class", 
    class_column="label", 
    tag2class=tag_to_label, 
    verbose=False
)
df_with_labels.head()

,class,filepath,label
0,c1d6f6c4,train/c1d6f6c4/bfeddcd4.jpg,0
1,c1d6f6c4,train/c1d6f6c4/bff0a9a0.jpg,0
2,c1d6f6c4,train/c1d6f6c4/bff1d1a4.jpg,0
3,c1d6f6c4,train/c1d6f6c4/bff3363e.jpg,0
4,c1d6f6c4,train/c1d6f6c4/bff4d7b4.jpg,0


And you should split data in `train/valid/test` parts.
There are only `train` and `valid` parts, so you must load test data as shows in a code cell.

In [ ]:
from catalyst.utils import split_dataframe_train_test

train_data, valid_data = split_dataframe_train_test(
    df_with_labels, test_size=0.2, random_state=42, stratify = df_with_labels['label'].values
)
train_data, valid_data = (
    train_data.to_dict("records"),
    valid_data.to_dict("records"),
)

In [ ]:
from catalyst.data.cv.reader import ImageReader
from catalyst.dl import utils
from catalyst.data import ScalarReader, ReaderCompose

num_classes = len(tag_to_label)

open_fn = ReaderCompose(
    [
        ImageReader(
            input_key="filepath", output_key="features", rootpath="train/"
        ),
        ScalarReader(
            input_key="label",
            output_key="targets",
            default_value=-1,
            dtype=np.int64,
        ),
        ScalarReader(
            input_key="label",
            output_key="targets_one_hot",
            default_value=-1,
            dtype=np.int64,
            one_hot_classes=num_classes,
        ),
    ]
)

## Augmentation

In a baseline, we don't have augmentation transformations in the baseline.
You can add them, if you expect that it will increase model accuracy.

In [ ]:
#AutoAugment - Learning Augmentation Policies from Data
#superior method for image augmentations
!git clone https://github.com/DeepVoltaire/AutoAugment.git
!mv /content/AutoAugment/autoaugment.py /content/autoaugment.py

In [ ]:
import albumentations as albu
from albumentations.pytorch import ToTensor
from autoaugment import ImageNetPolicy
from PIL import Image
from torchvision import transforms as trns


transform_to_tensor = albu.Compose(
    [   
     albu.Resize(224, 224),
     albu.Normalize(),
     ToTensor(),
    ]
)

transforms_imagenet = lambda image : {'image': trns.Compose([
                                       trns.ToPILImage(),
                                       trns.RandomResizedCrop(224),
                                       ImageNetPolicy(),
                                       trns.ToTensor(),
                                       trns.Normalize([0.485, 0.456, 0.406],
                                                      [0.229, 0.224, 0.225])])(image)}

transforms_grid_mask = albu.Compose(
    [
     albu.Resize(224, 224),
     albu.GridDropout(0.4, p = 1),
     albu.Normalize(),
     ToTensor()
    ]
)

own_transforms = albu.Compose(
    [albu.OneOf(
         [
          albu.RandomRotate90(p = 1),
          albu.HorizontalFlip(p = 1),
         ],
     p = 1),
     albu.Resize(224, 224),
     albu.Normalize(),
     ToTensor()
    ]
)
transformations = {'autoaugment': transforms_imagenet,
                   'to_tensor': transform_to_tensor,
                   'grid_mask': transforms_grid_mask,
                   'own_transforms': own_transforms}

In [ ]:
from catalyst.data import Augmentor

data_transforms = lambda transform: Augmentor(
    dict_key="features", augment_fn=lambda x: transformations[transform](image=x)["image"]
)


Don't forget creating test loader.

In [ ]:
batch_size = 16
num_workers = 4

train_loader = lambda augment: utils.get_loader(
    train_data,
    open_fn=open_fn,
    dict_transform=data_transforms(augment),
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=True,
    sampler=None,
    drop_last=True,
)

valid_loader = utils.get_loader(
    valid_data,
    open_fn=open_fn,
    dict_transform=data_transforms('to_tensor'),
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=False, 
    sampler=None,
    drop_last=True,
)

loaders = lambda augment: {
    "train": train_loader(augment),
    "valid": valid_loader
    }

## Model

For the baseline, we will use a ResNet model.
We already have examined in the seminar.
Enhance the model, use any* instruments or module as you like.

*(Don't forget about the rules!)

In [ ]:
from catalyst.contrib.nn.schedulers.onecycle import OneCycleLRWithWarmup
from torch.optim.lr_scheduler import CyclicLR, ReduceLROnPlateau
from catalyst.contrib.nn.optimizers.lookahead import Lookahead
from catalyst.contrib.nn.optimizers.radam import RAdam
from torch.nn import ReLU, PReLU, GELU, LeakyReLU
from catalyst.dl.callbacks import WandbLogger
from catalyst.core.callback import Callback
from collections import OrderedDict
from torch.optim import AdamW, SGD
import torch.nn.functional as F
from torch import Tensor

In [ ]:
class DenseLayer(nn.Module):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super().__init__()
        self.norm1 = nn.BatchNorm2d(num_input_features)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(num_input_features, bn_size *
                                           growth_rate, kernel_size=1, stride=1,
                                           bias=False)
        self.norm2 = nn.BatchNorm2d(bn_size * growth_rate)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(bn_size * growth_rate, growth_rate,
                                           kernel_size=3, stride=1, padding=1,
                                           bias=False)
        self.drop_rate = float(drop_rate)

    def bn_function(self, inputs):
        concated_features = torch.cat(inputs, 1)
        bottleneck_output = self.conv1(self.relu1(self.norm1(concated_features)))
        return bottleneck_output

    def forward(self, input):
        if isinstance(input, Tensor):
            prev_features = [input]
        else:
            prev_features = input

        bottleneck_output = self.bn_function(prev_features)
        new_features = self.conv2(self.relu2(self.norm2(bottleneck_output)))
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate,
                                     training=self.training)
        return new_features


class DenseBlock(nn.ModuleDict):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super().__init__()
        for i in range(num_layers):
            layer = DenseLayer(
                num_input_features + i * growth_rate,
                growth_rate=growth_rate,
                bn_size=bn_size,
                drop_rate=drop_rate
            )
            self.add_module('denselayer%d' % (i + 1), layer)

    def forward(self, init_features):
        features = [init_features]
        for name, layer in self.items():
            new_features = layer(features)
            features.append(new_features)
        return torch.cat(features, 1)


class Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super().__init__()
        self.add_module('norm', nn.BatchNorm2d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2))


class DenseNet(nn.Module):
    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0, num_classes=10):

        super().__init__()

        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(3, num_init_features, kernel_size=7, stride=2,
                                padding=3, bias=False)),
            ('norm0', nn.BatchNorm2d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),
        ]))

        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = DenseBlock(
                num_layers=num_layers,
                num_input_features=num_features,
                bn_size=bn_size,
                growth_rate=growth_rate,
                drop_rate=drop_rate
            )
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = Transition(num_input_features=num_features,
                                    num_output_features=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        self.features.add_module('norm5', nn.BatchNorm2d(num_features))
        self.classifier = nn.Linear(num_features, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out

In [ ]:
# Create Runner and train your model!

class Runner(dl.Runner):
    def predict_batch(self, batch):
        return self.model(batch[0].to(self.device))

    def _handle_batch(self, batch):
        y_pred = self.model(batch["features"])

        self.input = batch
        self.output = {"logits": y_pred}

In [ ]:
from torch_lr_finder import LRFinder, TrainDataLoaderIter


class TrainLoader(TrainDataLoaderIter):
    def __init__(self, data_loader, auto_reset=True):
        super().__init__(data_loader)

    def inputs_labels_from_batch(self, batch_data):
        inputs, labels, *_ = batch_data.values()

        return inputs, labels

def find_lr(model, criterion, optimizer, augment):
  loader = TrainLoader(loaders(augment)['train'])
  if optimizer == 'sgd':
    optimizer = torch.optim.SGD(model.parameters(), 0.01)
  elif optimizer == 'adamW':
    optimizer = AdamW(model.parameters())
  elif optimizer == 'Radam':
    optimizer = RAdam(model.parameters())
  elif optimizer == 'adam':
    optimizer = torch.optim.Adam(model.parameters())
  elif optimizer == 'lookahead':
    optimizer = Lookahead(AdamW(model.parameters()))

  lr_finder = LRFinder(model, optimizer, criterion, device="cuda")
  lr_finder.range_test(loader, end_lr=1, num_iter=100)

  losses = np.array(lr_finder.history['loss'])
  lrs = lr_finder.history['lr']
  lr_finder.plot()
  lr_finder.reset()
  #find 3 lrs: optimal and lrs for cycliclr scheduler
  max_lr = lrs[losses.argmin()]
  min_lr = lrs[losses[:losses.argmin()].argmax()]
  optimal = lrs[np.gradient(losses).argmin()]
  return optimal, min_lr, max_lr

In [ ]:
from tqdm import tqdm
import wandb
import optuna


def model_train(activation, augment, optimizer_to_choose, scheduler_to_choose, lr, min_lr, max_lr):
  model = DenseNet(48, (6, 12, 36, 24), 96)

  criterion = nn.CrossEntropyLoss()
  epoch = 60
  optimizers = {'adamW': lambda x: AdamW(model.parameters(), lr = x),
              'adam': lambda x: torch.optim.Adam(model.parameters(), lr = x),
              'Radam': lambda x: RAdam(model.parameters(), lr = x),
              'lookahead': lambda x: Lookahead(AdamW(model.parameters(), lr = x)),
              'sgd': lambda x: torch.optim.SGD(model.parameters(), lr = x)}
  optimizer = optimizers[optimizer_to_choose](lr)
  schedulers = {'cyclic_lr': lambda x: CyclicLR(x, min_lr, max_lr, step_size_up=1),
              'one_cycle': lambda x: OneCycleLRWithWarmup(x, num_steps = epoch, warmup_steps=2, lr_range=(max_lr, min_lr)),
              'reduce_onplatue': lambda x: ReduceLROnPlateau(x, patience=2, factor = 0.5)}
  scheduler = schedulers[scheduler_to_choose](optimizer)
  config = {'optimizer': optimizer,
          'batch_size': batch_size,
          'scheduler': scheduler,
          'epoch': epoch,
          'augment': augment}

  runner = Runner()
  runner.train(
      model=model,
      optimizer=optimizer,
      criterion=criterion,
      scheduler = scheduler,
      loaders=loaders(augment),
      logdir=Path("logs") / datetime.now().strftime("%Y%m%d-%H%M%S"),
      num_epochs=config['epoch'],
      verbose=True,
      load_best_on_end=True,
      main_metric="loss",
      minimize_metric = True,
      callbacks={
          "optimizer": dl.OptimizerCallback(
              metric_key="loss", accumulation_steps=1, grad_clip_params=None,
          ),
          "criterion": dl.CriterionCallback(
              input_key="targets", output_key="logits", prefix="loss",
          ),
          "accuracy": dl.AccuracyCallback(num_classes=10),
          "scheduler": dl.SchedulerCallback(),
          'Wandb': WandbLogger(
                          project="imagenette_classification",
                          name = f"model_{optimizer_to_choose}_{scheduler_to_choose}_{augment}_{activation}",
                          config = config
                          )
      },
  )
  return runner, model


def objective(trial):
    lr = trial.suggest_loguniform("lr", 1e-4, 1e-3)
    activation = trial.suggest_categorical('activation', ['gelu'])
    augment = trial.suggest_categorical('augment', ['autoaugment', 'to_tensor'])
    min_lr = trial.suggest_loguniform("min_lr", 1e-5, 1e-3)
    max_lr = trial.suggest_loguniform("max_lr", 1e-3, 1e-2)
    optimizer = trial.suggest_categorical('optimizer', ['Radam'])
    scheduler = trial.suggest_categorical('scheduler', ['one_cycle'])

    runner, _ = model_train(trial, activation, augment, optimizer, scheduler, lr, min_lr, max_lr)
    return runner.best_valid_metrics[runner.main_metric]

In [ ]:
runner, model = model_train('prelu', 'autoaugment', 'Radam', 'reduce_onplatue', 1e-2, 1e-4, 1e-2)

In [ ]:
tqdm._instances.clear()

Test transforms are crucial in order to boost accuracy

In [ ]:
import albumentations as albu
from albumentations.pytorch import ToTensor
import ttach as tta


transform_to_tensor = albu.Compose(
    [
        albu.Normalize(),
        ToTensor(),
    ]
)
transforms_list = { 
    'original': albu.Compose([
        albu.Resize(224, 224),
    ]),   
    'crop_180': albu.Compose([
        albu.CenterCrop(180, 180),
        albu.Resize(224, 224),
    ]),    
    'crop_160': albu.Compose([
        albu.CenterCrop(160, 160),
        albu.Resize(224, 224),
    ]),
    'gray_200': albu.Compose([
        albu.ToGray(p = 1),
        albu.CenterCrop(200, 200),
        albu.Resize(224, 224),
    ]),
    'r_crop_180_1': albu.Compose([
        albu.RandomCrop(180, 180),
        albu.Resize(224, 224),
    ]),
    'r_crop_180_2': albu.Compose([
        albu.RandomCrop(180, 180),
        albu.Resize(224, 224),
    ]),
    'r_crop_180_3': albu.Compose([
        albu.ToGray(p = 1),
        albu.RandomCrop(180, 180),
        albu.Resize(224, 224),
    ]),  
    'rotate_90': albu.Compose([
        albu.RandomRotate90(p = 1),
        albu.Resize(224, 224)
    ])    
}

This code below will generate a submission.
It reads images from `test` folder and gathers prediction from the trained model.
Check your submission before uploading it into `Kaggle`.

In [ ]:
from PIL import Image
from tqdm import tqdm

submission = {"Id": [], "Category": []}
model.eval()

for file in tqdm(Path("test").iterdir(), total = len(os.listdir('test'))):
    image = np.array(Image.open(file).convert('RGB'))
    img = transforms_list['original'](image = image)['image']
    input = transform_to_tensor(image = img)['image']
    input = input.unsqueeze(0).cuda()
    features = model(input)
    probabilities = torch.softmax(features, -1)
    
    pred = probabilities.detach().cpu().numpy()
    pred = np.argmax(pred)
    submission["Id"].append(file.name[:-4])
    submission["Category"].append(class_names[pred])

100%|██████████| 3925/3925 [02:41<00:00, 24.24it/s]


In [ ]:
import pandas as pd


pd.DataFrame(submission).to_csv("baseline.csv", index=False)

In [ ]:
pd.DataFrame(submission)

